# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import AdaBoostClassifier
import pickle
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BMWPC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BMWPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BMWPC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('clean_df', engine) 
X = df['message']
Y = df.iloc[:, 4:] 

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [3]:
#  check if  message contains string for http
bool = X.str.contains('http\.')
filter_X = X[bool]
filter_X.size

0

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()  ) 

    tokens = word_tokenize(text)    

    # Remove stop words
    words = [w for w in tokens if w not in stopwords.words("english")]        

    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]

    # Lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]    

    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in lemmed]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
forest = RandomForestClassifier(n_estimators = 50, random_state = 1) 

# PRELIMINARY PIPELINE
pipeline = Pipeline([
                     ('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(forest))
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
predicted = pipeline.predict(X_test)

In [15]:
for i, column in enumerate(Y_test.columns):
    print('Test results for "' + column + '" :\n')
    print(classification_report(Y_test[column], pd.DataFrame(predicted)[i]))
    print('\n\n')

Test results for "related" :

              precision    recall  f1-score   support

           0       0.68      0.42      0.52      1202
           1       0.84      0.94      0.89      4000
           2       0.34      0.48      0.40        42

    accuracy                           0.81      5244
   macro avg       0.62      0.61      0.60      5244
weighted avg       0.80      0.81      0.80      5244




Test results for "request" :

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4333
           1       0.86      0.48      0.61       911

    accuracy                           0.90      5244
   macro avg       0.88      0.73      0.78      5244
weighted avg       0.89      0.90      0.88      5244




Test results for "offer" :

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5221
           1       0.00      0.00      0.00        23

    accuracy                      

E:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)





Test results for "tools" :

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5207
           1       0.00      0.00      0.00        37

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      0.99      0.99      5244




Test results for "hospitals" :

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5190
           1       0.00      0.00      0.00        54

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.98      0.99      0.98      5244




Test results for "shops" :

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5211
           1       0.00      0.00      0.00        33

    accuracy                           0.99      5244
   macro avg       0.50      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
parameters = {
    'cvect__min_df': [2, 4],
    'tfidf__use_idf':[True, False],
    'clf__estimator__n_estimators':[10, 50],
    'clf__estimator__min_samples_split':[2, 4, 8]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10,
                  n_jobs=4)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  6.3min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  9.4min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed: 17.9min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 22.1min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 31.2min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 39.3min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed: 49.3min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 56.4min
[Parallel(n_jobs=4)]: Done  72 out of  72 | elapsed: 64.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('cvect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [7]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [8]:
new_pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('cvect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters2 = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000),
    'features__text_pipeline__tfidf__use_idf': (True, False),
#    'clf__n_estimators': [10, 100],
#    'clf__learning_rate': [0.01, 0.1],
#    'features__transformer_weights': (
#        {'text_pipeline': 1, 'starting_verb': 0.5},
#        {'text_pipeline': 0.5, 'starting_verb': 1},
#        {'text_pipeline': 0.8, 'starting_verb': 1},
#    )
}

cv2 = GridSearchCV(new_pipeline, param_grid=parameters2, verbose=10,
                  n_jobs=4)

In [10]:
cv2.fit(X_train, Y_train)

E:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=4)]: Batch computation too fast (0.1690s.) Setting batch_size=2.


PicklingError: Could not pickle the task to send it to the workers.

### 9. Export your model as a pickle file

In [29]:
# save the model to disk
filename = 'model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.